## Loading Libraries

In [1]:
import pandas as pd
import numpy as np
import torch
import evaluate

c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = torch.device('cuda')

In [3]:
x = torch.randn(30)
y = torch.randn(30)

z = torch.cat([x.reshape(-1,1), y.reshape(-1,1)], 1)
print(x.shape, y.shape, z.shape)

torch.Size([30]) torch.Size([30]) torch.Size([30, 2])


## Loading Dataset

In [4]:
df = pd.read_csv('../data/processed/train_sample_processed.csv')

In [5]:
statuses = np.unique(df['OpenStatus'].values)

id2label = {idx: label for idx, label in enumerate(statuses)}
label2id = {label: idx for idx, label in enumerate(statuses)}

In [6]:

from custom_dataset import GithubDataset
from torch.utils.data import random_split
from datasets import Dataset

# Create the pytorch dataset
full_dataset = GithubDataset(df)

In [37]:
from collections import Counter

full_dataset[0]
ds_dict = 
train_classes = [label['status'] for label in full_dataset]
min_size = min(Counter(train_classes).values())
# train_classes

6158

In [38]:
full_dataset[0]

{'tags_onehot': tensor([False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False,
         False, False, False, False, False, False, False, False, False, False]),
 'unrecognized_tags_count': tensor(5),
 'reputation': tensor(1),
 'undeleted_answers': tensor(2),
 'user_life_days': tensor(243),
 'title': 'For Mongodb is it better to reference an object or u

In [8]:
train_dataset, validation_dataset, test_dataset = random_split(full_dataset, [0.7, 0.25, 0.05])

train_dataset = Dataset.from_dict(train_dataset[:])
validation_dataset = Dataset.from_dict(validation_dataset[:])
test_dataset = Dataset.from_dict(test_dataset[:])

In [9]:
# renaming_dict = {"text_content": "text", "status": "label"}
renaming_dict = {"status": "labels"}

train_dataset = train_dataset.rename_columns(renaming_dict)
validation_dataset = validation_dataset.rename_columns(renaming_dict)
test_dataset = test_dataset.rename_columns(renaming_dict)

### Tokenizing Data

In [10]:
from tokenizers import normalizers
from tokenizers.normalizers import NFD, StripAccents

# We create our normalizer which will appy Unicode normalization and strip accents
normalizer = normalizers.Sequence([NFD(), StripAccents()])

normalizer.normalize_str("Héllò? What aré yòü üptò tòday?")

from tokenizers.pre_tokenizers import Whitespace

# We create our pre-tokenizer which will split based on the regex \w+|[^\w\s]+
pre_tokenizer = Whitespace()

In [11]:
from tokenizers.models import WordPiece
from tokenizers import Tokenizer

tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = normalizer
tokenizer.pre_tokenizer = pre_tokenizer

In [12]:
from tokenizers.trainers import WordPieceTrainer
import time

# We will create a batch iterator which will generate a batch of sentences for training
# our tokenizaer. This is the preferred way instead of passing single sentences to the
# tokenizer as it will a lot faster.
def batch_iterator(dataset, batch_size=10000):
  for i in range(0, len(dataset), batch_size):
    lower_idx = i
    # Ensure the upper idx doesn't overflow leading to an 'IndexError'
    upper_idx = i + batch_size if i + batch_size <= len(dataset) else len(dataset)
    text = dataset[lower_idx : upper_idx]["title"] + ' ' + dataset[lower_idx : upper_idx]["text_content"]
    yield text
        
# We pass in the list of special tokens so that our model knows about them.
trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

tic = time.perf_counter()
# Now, we do batch training based on our iterator that we defined earlier.
tokenizer.train_from_iterator(batch_iterator(full_dataset), trainer=trainer, length=len(full_dataset))
toc = time.perf_counter()
print(f"Elapsed time: {toc - tic:0.4f} seconds")

Elapsed time: 9.4849 seconds


In [13]:
label2id
[label2id[label] for label in train_dataset['labels'][:10]]

[1, 3, 3, 1, 0, 1, 3, 2, 3, 1]

In [14]:
CONTENT_MAX_TEXT_CONTENT = 128
TITLE_MAX_TEXT_CONTENT = 32

def tokenize_func(batch):
  tokenized_batch = dict()
  
  tokenized_batch['labels'] = [label2id[label] for label in batch['labels']]
  
  tokenizer.enable_truncation(max_length=CONTENT_MAX_TEXT_CONTENT)
  tokenizer.enable_padding(length=CONTENT_MAX_TEXT_CONTENT)
  content_out = tokenizer.encode_batch(batch['text_content'])
  
  tokenizer.enable_truncation(max_length=TITLE_MAX_TEXT_CONTENT)
  tokenizer.enable_padding(length=TITLE_MAX_TEXT_CONTENT)
  title_out = tokenizer.encode_batch(batch['title'])
  
  tokenized_batch['content_input_ids'] = torch.IntTensor(list(map(lambda x: x.ids, content_out)))
  tokenized_batch['title_input_ids'] = torch.IntTensor(list(map(lambda x: x.ids, title_out)))

  return tokenized_batch

In [15]:
tokenized_train_dataset = train_dataset.map(tokenize_func, batched=True)
tokenized_validation_dataset = validation_dataset.map(tokenize_func, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_func, batched=True)

Map: 100%|██████████| 7013/7013 [00:00<00:00, 11401.65 examples/s]


In [16]:
tokenized_train_dataset

Dataset({
    features: ['tags_onehot', 'unrecognized_tags_count', 'reputation', 'undeleted_answers', 'user_life_days', 'title', 'text_content', 'labels', 'content_input_ids', 'title_input_ids'],
    num_rows: 98191
})

In [17]:
columns_to_remove = ['title', 'text_content']

tokenized_train_dataset = tokenized_train_dataset.remove_columns(columns_to_remove)
tokenized_validation_dataset = tokenized_validation_dataset.remove_columns(columns_to_remove)
tokenized_test_dataset = tokenized_test_dataset.remove_columns(columns_to_remove)

#### COMMENT - IDEAS

We probably should:
- retrain the whole model (probably smaller) with
- better tokenizer - built up from the ground including all the names of the specific tech (languages, frameworks, IDEs, etc.)

## Model - Custom

In [18]:
from custom_model import AutoCompositeModel, TagsModel, NumericalPartModel, TextualPartModel

model_tags = TagsModel(device).to(device)

model_numerical = NumericalPartModel(device).to(device)

model_textual = TextualPartModel(device).to(device)

## Training Setup

### Data Loaders

In [19]:
def collate_func(batch):
  fixed_batch = pd.DataFrame(batch).to_dict(orient="list")
  # print(fixed_batch)

  return fixed_batch

In [20]:
from torch.utils.data import DataLoader
from sampler import StratifiedBatchSampler
from training_own import to_my_tensor

BATCH_SIZE = 16
training_sampler = StratifiedBatchSampler(to_my_tensor(tokenized_train_dataset['labels'], device), BATCH_SIZE)
training_loader = DataLoader(tokenized_train_dataset, batch_sampler=training_sampler, collate_fn=collate_func)

validation_sampler = StratifiedBatchSampler(to_my_tensor(tokenized_validation_dataset['labels'], device), BATCH_SIZE)
validation_loader = DataLoader(tokenized_validation_dataset, batch_sampler=validation_sampler)

test_sampler = StratifiedBatchSampler(to_my_tensor(tokenized_test_dataset['labels'], device), BATCH_SIZE)
test_loader = DataLoader(tokenized_test_dataset, batch_sampler=test_sampler)

In [21]:
print(tokenized_train_dataset['title_input_ids'][0])
print(len(tokenized_train_dataset['title_input_ids'][0]))

[3209, 3060, 45, 3775, 69, 3907, 12, 13160, 18557, 3459, 13, 4136, 2991, 2996, 5598, 3101, 16386, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
32


### Loss Function

In [22]:
loss_fn = torch.nn.CrossEntropyLoss()

### Optimizer

In [23]:
# Optimizers specified in the torch.optim package
# optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

# optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-5, momentum=0.8)


optimizer_tags = torch.optim.Adam(model_tags.parameters(), lr=1e-5)
optimizer_numerical = torch.optim.Adam(model_numerical.parameters(), lr=1e-4)
optimizer_textual = torch.optim.Adam(model_textual.parameters(), lr=1e-3)

In [24]:
from torchmetrics import Accuracy

accuracy_metric = Accuracy(task='multiclass', num_classes=5).to(device)

### Trainer Setup

In [25]:
from training_own import Trainer, TrainerConfiguration, get_model_params

In [26]:
config_tags = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer_tags,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)
trainer_tags = Trainer(model=model_tags, trainer_configuration=config_tags, input_columns=['tags_onehot'], output_column='labels')

In [27]:
config_numerical = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer_tags,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)
trainer_numerical = Trainer(model=model_numerical, trainer_configuration=config_numerical, input_columns=['unrecognized_tags_count', 'reputation', 'undeleted_answers', 'user_life_days'], output_column='labels')

In [28]:
config_textual = TrainerConfiguration(
  training_loader=training_loader,
  validation_loader=validation_loader,
  optimizer=optimizer_textual,
  loss_fn=loss_fn,
  accuracy_metric=accuracy_metric,
  device=device
)
trainer_textual = Trainer(model=model_textual, trainer_configuration=config_textual, input_columns=['content_input_ids', 'title_input_ids'], output_column='labels')

In [29]:
print(get_model_params(model_tags), get_model_params(model_numerical), get_model_params(model_textual))

9121 437 744809


In [30]:
from training_own import to_my_tensor

X = to_my_tensor([to_my_tensor(tokenized_train_dataset[k], 'cpu') for k in ['unrecognized_tags_count', 'reputation', 'undeleted_answers', 'user_life_days']], 'cpu')
# y = dict((k, to_my_tensor(train_dataset[k], device)) for k in ['labels'])
y = tokenized_train_dataset['labels']
# X

In [31]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

model_clasical = KNeighborsClassifier(n_neighbors=50)
# model_clasical = SVC(kernel='linear')

model_clasical.fit(X, y)

KNeighborsClassifier(n_neighbors=50)

In [32]:
# X = to_my_tensor([to_my_tensor(tokenized_test_dataset[k], 'cpu') for k in ['unrecognized_tags_count', 'reputation', 'undeleted_answers', 'user_life_days']], 'cpu')
# y = tokenized_test_dataset['labels']

# model_clasical.score(X, y)

In [33]:
# trainer_tags.train_many_epochs(epochs=2, logging_frequency=100, evaluate_when_logging=False)

In [34]:
# trainer_numerical.train_many_epochs(epochs=4, logging_frequency=100, evaluate_when_logging=False)

In [35]:
from training_own import to_my_tensor

input = dict((k, to_my_tensor(test_dataset[k], device)) for k in ['unrecognized_tags_count', 'reputation', 'undeleted_answers', 'user_life_days'])
res = model_numerical(input)
# test_dataset[0]

res[4]

tensor([-4.3716,  2.2147,  1.2273, -5.4561, -4.6778], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [36]:
trainer_textual.train_many_epochs(epochs=2, logging_frequency=100, evaluate_when_logging=False)

EPOCH 1:


c:\Users\Pawel\anaconda3\envs\env_torch\lib\site-packages\sklearn\model_selection\_split.py:725: UserWarning: The least populated class in y has only 4316 members, which is less than n_splits=6136.
  warnings.warn(


 batch 100 training_loss: 442.79836959838866 training_accuracy: 0.35753676295280457
 batch 200 training_loss: 131.9523091506958 training_accuracy: 0.35874998569488525
 batch 300 training_loss: 40.559086303710934 training_accuracy: 0.36249998211860657
 batch 400 training_loss: 7.237912255525589 training_accuracy: 0.38374999165534973
 batch 500 training_loss: 2.63902570605278 training_accuracy: 0.38499999046325684
 batch 600 training_loss: 2.0701826882362364 training_accuracy: 0.4156249761581421
 batch 700 training_loss: 1.8196184408664704 training_accuracy: 0.42124998569488525
 batch 800 training_loss: 1.9143447995185852 training_accuracy: 0.4256249964237213
 batch 900 training_loss: 1.492272639274597 training_accuracy: 0.4443749785423279
 batch 1000 training_loss: 1.6352219903469085 training_accuracy: 0.46812498569488525
 batch 1100 training_loss: 1.3962281680107116 training_accuracy: 0.4712499976158142
 batch 1200 training_loss: 1.422214435338974 training_accuracy: 0.4806250035762787


KeyboardInterrupt: 